<a href="https://colab.research.google.com/github/harshit4311/Pepsi-Coke-LogoDetection/blob/main/Pre_Trained_Model(YOLOv8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install ultralytics
!pip install av
!pip install opencv-python

import av
import zipfile
import os
import json
import numpy as np
from ultralytics import YOLO

print("Installation done")

Installation done


In [13]:
# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

In [5]:
# Define custom classes (Pepsi and CocaCola)
model.classes = ['Pepsi', 'CocaCola']


In [ ]:
# Load pre-trained YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Modify the model for custom classes (Pepsi and CocaCola)
model.classes = ['Pepsi', 'CocaCola']

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-7-15 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 131MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [7]:
import zipfile
import os

# Extract zip files
with zipfile.ZipFile('/content/coke-dataset-photos.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/coke-dataset')

with zipfile.ZipFile('/content/pepsi-dataset-photos.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/pepsi-dataset')

# Define paths to datasets
coke_path = '/content/coke-dataset'
pepsi_path = '/content/pepsi-dataset'

In [8]:
# Function to extract frames
def extract_frames(video_path, interval=1):
    container = av.open(video_path)
    frames = []
    timestamps = []
    for frame in container.decode(video=0):
        # Extract frames based on the specified interval
        if frame.time >= len(timestamps) * interval:
            frames.append(frame.to_image())
            timestamps.append(frame.time)
    return frames, timestamps

In [9]:
# Function to detect logos
def detect_logos(model, frames, timestamps):
    pepsi_timestamps = []
    cocacola_timestamps = []

    for i, frame in enumerate(frames):
        frame = np.array(frame)  # Convert PIL image to numpy array
        results = model(frame)  # Get predictions from the model

        for result in results:
            # Loop through detected objects
            for pred in result.boxes:
                # The format of pred is [x1, y1, x2, y2, confidence, class]
                class_id = int(pred.cls)
                if class_id == 0:  # Assuming class 0 is Pepsi
                    pepsi_timestamps.append(timestamps[i])
                elif class_id == 1:  # Assuming class 1 is CocaCola
                    cocacola_timestamps.append(timestamps[i])

    return pepsi_timestamps, cocacola_timestamps

In [10]:
# Function to save results in a JSON file
def save_results(pepsi_timestamps, cocacola_timestamps, output_path='results.json'):
    results = {
        "Pepsi_pts": pepsi_timestamps,
        "CocaCola_pts": cocacola_timestamps
    }
    with open(output_path, 'w') as f:
        json.dump(results, f)


In [15]:
video_path = '/content/Coke_Pepsi_Video.mp4'
frames, timestamps = extract_frames(video_path)
pepsi_timestamps, cocacola_timestamps = detect_logos(model, frames, timestamps)
save_results(pepsi_timestamps, cocacola_timestamps)


0: 384x640 (no detections), 862.3ms
Speed: 5.0ms preprocess, 862.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bottles, 2 cups, 1 dining table, 385.4ms
Speed: 4.6ms preprocess, 385.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bottles, 2 cups, 1 dining table, 394.0ms
Speed: 3.6ms preprocess, 394.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 bottles, 2 cups, 392.3ms
Speed: 3.9ms preprocess, 392.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 bottles, 1 cup, 1 refrigerator, 387.7ms
Speed: 5.2ms preprocess, 387.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 bottle, 2 cups, 1 dining table, 392.5ms
Speed: 3.6ms preprocess, 392.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bottle, 1 wine glass, 402.4ms
Speed: 4.3ms preprocess, 402.4ms inference